## <font color='darkblue'>Generate Mocking Data</font>

In [1]:
from keras.datasets.mnist import load_data
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# example of defining the discriminator model
from keras.models import Sequential
from keras.models import load_model
from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import Reshape
from keras.layers import Conv2DTranspose

from keras.utils.vis_utils import plot_model

# example of training the discriminator model on real and random mnist images
from numpy import expand_dims
from numpy import ones
from numpy import zeros
from numpy import vstack
from numpy.random import rand
from numpy.random import randint
from numpy.random import randn
from numpy import asarray

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=PendingDeprecationWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

TEST_CASE_LIST = (
  'A', 'B', 'C', 'D', 'E', 'F', 'G')
NUM_OF_TEST_CASE = len(TEST_CASE_LIST)
TEST_CASE_SEQ_LENGTH = 5

2023-07-05 06:56:35.550947: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-05 06:56:35.619798: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-05 06:56:35.621304: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 06:56:36.801328: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def one_hot_encoding(test_case: str):
  try:
    test_case_pos = TEST_CASE_LIST.index(test_case)
  except ValueError as ex:
    raise ex
  encoding = [0] * NUM_OF_TEST_CASE
  encoding[test_case_pos] = 1
  return encoding


def raw_data_2_test_case_sequence(raw_data, debug=False):
  raw_data = raw_data.reshape(5, 7)
  if debug:
    print(f'raw_data ({raw_data.shape}):\n{raw_data}')
    print(f'argmax:\n{np.argmax(raw_data, axis=1)}')
  test_case_sequence = []
  for test_case_pos in np.argmax(raw_data, axis=1):
      test_case_sequence.append(TEST_CASE_LIST[test_case_pos])
      
  return test_case_sequence


def test_case_sequence_2_df(test_case_seq_list: list[str], labels: list[int] = None):
  test_data = []
  for test_case_seq in test_case_seq_list:
    encoded_test_case_seuqnece = []
    for test_case in test_case_seq:
      encoded_test_case_seuqnece.append(one_hot_encoding(test_case))
      
    test_data.append(encoded_test_case_seuqnece)

  X = np.array(test_data)
  #print(f'data shape={np_data.shape}\n{np_data}')
  # np_data = np.reshape(np_data, (len(test_case_seq_list), TEST_CASE_SEQ_LENGTH * NUM_OF_TEST_CASE))
  #X = pd.DataFrame(np_data)
  y = np.array(labels) if labels else None
  return X, y

In [3]:
cr_raw_data_list = [
  ('A', 'B', 'C', 'E', 'G'),
  ('C', 'F', 'A', 'F', 'D'),
  ('C', 'A', 'D', 'G', 'C'),
  ('B', 'D', 'C', 'A', 'F'),
  ('C', 'F', 'D', 'A', 'G'),
  ('A', 'B', 'C', 'E', 'G'),
  ('D', 'A', 'G', 'A', 'B'),
  ('D', 'E', 'G', 'A', 'B'),
  ('G', 'A', 'D', 'C', 'F'),
  ('F', 'D', 'A', 'B', 'C'),
  ('A', 'B', 'C', 'E', 'G'),
  ('C', 'F', 'D', 'A', 'B'),
  ('C', 'F', 'D', 'E', 'G'),
  ('A', 'B', 'C', 'E', 'G'),
  ('D', 'E', 'G', 'A', 'B'),
  ('F', 'D', 'C', 'A', 'B'),
  ('C', 'F', 'D', 'E', 'G'),
  ('A', 'B', 'C', 'E', 'G'),
  ('D', 'E', 'G', 'A', 'B'),
  ('E', 'G', 'C', 'A', 'B'),
  ('G', 'A', 'D', 'C', 'F'),
  ('F', 'D', 'C', 'F', 'B'),
  ('A', 'B', 'C', 'A', 'G'),
  ('D', 'E', 'G', 'D', 'A'),
  ('C', 'A', 'B', 'C', 'B'),
  ('F', 'A', 'B', 'C', 'F'),
  ('E', 'G', 'B', 'C', 'F'),
]

miss_raw_data_list = [
  ('B', 'C', 'D', 'G', 'A'),
  ('C', 'C', 'D', 'D', 'B'),
  ('A', 'D', 'D', 'F', 'B'),
  ('G', 'G', 'A', 'A', 'C'),
  ('B', 'C', 'F', 'A', 'E'),
  ('A', 'A', 'B', 'B', 'G'),
  ('E', 'A', 'D', 'C', 'G'),
  ('D', 'D', 'B', 'A', 'A'),
  ('F', 'F', 'B', 'C', 'E'),
  ('F', 'A', 'A', 'G', 'G'),
  ('E', 'D', 'D', 'D', 'E'),
  ('B', 'B', 'B', 'C', 'C'),
  ('G', 'D', 'D', 'E', 'E'),
  ('C', 'C', 'C', 'A', 'A'),
  ('A', 'A', 'A', 'E', 'E'),
  ('G', 'G', 'F', 'A', 'A'),
  ('B', 'A', 'A', 'A', 'B'),
  ('F', 'C', 'C', 'A', 'A'),
  ('C', 'C', 'A', 'A', 'C'),
  ('E', 'A', 'C', 'C', 'E'),
  ('E', 'B', 'B', 'E', 'B'),
  ('A', 'A', 'A', 'E', 'E'),
  ('B', 'B', 'B', 'A', 'F'),
  ('F', 'G', 'D', 'D', 'D'),
  ('G', 'G', 'G', 'C', 'E'),
  ('G', 'G', 'G', 'C', 'C'),
  ('B', 'G', 'G', 'C', 'C'),
  ('G', 'C', 'C', 'C', 'F'),
  ('D', 'A', 'D', 'A', 'A'),
  ('D', 'G', 'D', 'G', 'G'),
  ('F', 'F', 'C', 'G', 'C'),
  ('F', 'F', 'F', 'C', 'G'),
  ('A', 'G', 'G', 'G', 'F'),
  ('E', 'G', 'G', 'G', 'G'),
  ('A', 'A', 'A', 'F', 'E'),
  ('E', 'G', 'G', 'G', 'F'),
  ('A', 'A', 'A', 'A', 'A'),
  ('G', 'G', 'G', 'G', 'F'),
  ('F', 'A', 'B', 'B', 'B'),
  ('A', 'B', 'B', 'B', 'A'),
  ('G', 'G', 'G', 'G', 'G'),
]

test_case_list = cr_raw_data_list
labels = [1] * len(cr_raw_data_list) + [0] * len(miss_raw_data_list)

In [4]:
X, y = test_case_sequence_2_df(test_case_list, labels)
X.shape

(27, 5, 7)

In [5]:
X[:1]

array([[[1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1]]])

## <font color='darkblue'>Define and Train the Discriminator Model</font>
<b>The model must take a sample input test case sequence from our dataset as input and output a classification prediction as to whether the sample is real or fake.</b> This is a binary classification problem:
* **Inputs**: Image with one channel and 5x7 shape (Sequence length=5; test case number=7).
* **Outputs**: Binary classification, likelihood the sample is real (or fake).

In [6]:
# define the standalone discriminator model
def define_discriminator(in_shape=(5, 7, 1)):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.01))
    model.add(Dropout(0.01))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    # compile model
    opt = Adam(lr=0.01, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [7]:
# define model
model = define_discriminator()

# summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3, 4, 64)          640       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 3, 4, 64)          0         
                                                                 
 dropout (Dropout)           (None, 3, 4, 64)          0         
                                                                 
 dropout_1 (Dropout)         (None, 3, 4, 64)          0         
                                                                 
 flatten (Flatten)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 1)                 769       
                                                                 
Total params: 1,409
Trainable params: 1,409
Non-trainabl

/usr/local/google/home/johnkclee/Gitrepos/ml_articles/env/lib/python3.11/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [8]:
X_train = np.expand_dims(X, axis=-1)

In [9]:
X_train.shape

(27, 5, 7, 1)

In [10]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y

In [11]:
X, _ = generate_real_samples(X_train, 1)
raw_data_2_test_case_sequence(X[0])

['F', 'D', 'C', 'A', 'B']

In [12]:
miss_raw_data_ary, _ = test_case_sequence_2_df(miss_raw_data_list)
miss_raw_data_ary[0].reshape(TEST_CASE_SEQ_LENGTH * NUM_OF_TEST_CASE)

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0])

In [13]:
# generate n fake samples with class labels
def generate_fake_samples(n_samples):
    half_random_num = n_samples // 3 if n_samples > 3 else 0
    # 1) generate uniform random numbers in [0,1]
    X1 = randint(2, size=TEST_CASE_SEQ_LENGTH * NUM_OF_TEST_CASE * half_random_num)
    #print(f'X1 ({X1.shape}):\n{X1}')
    
    
    # 2) sampling from miss_raw_data_list
    rest_fake_num = n_samples - half_random_num
    if rest_fake_num > 0:
      ix = randint(0, len(miss_raw_data_list), rest_fake_num)
      miss_raw_data_ary, _ = test_case_sequence_2_df(miss_raw_data_list)
      X2 = miss_raw_data_ary[ix].reshape(TEST_CASE_SEQ_LENGTH * NUM_OF_TEST_CASE * rest_fake_num)
      #print(f'X2 ({X2.shape}):\n{X2}')
      X = np.array(X1.tolist() + X2.tolist())
    else:
      X = X1
    
    # print(f'X {X.shape}:\n{X}')
    X = X.reshape((n_samples, TEST_CASE_SEQ_LENGTH, NUM_OF_TEST_CASE, 1))
    # generate 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

In [14]:
fake_X, _ = generate_fake_samples(3)
raw_data_2_test_case_sequence(fake_X[0])

['E', 'B', 'B', 'E', 'B']

In [15]:
# train the discriminator model
def train_discriminator(model, dataset, n_iter=100, n_batch=256):
    real_batch = int(n_batch / 3)
    fake_batch = n_batch - real_batch
    # manually enumerate epochs
    for i in range(n_iter):
        # get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(dataset, real_batch)
        # update discriminator on real samples
        _, real_acc = model.train_on_batch(X_real, y_real)
        # generate 'fake' examples
        X_fake, y_fake = generate_fake_samples(fake_batch)
        # update discriminator on fake samples
        _, fake_acc = model.train_on_batch(X_fake, y_fake)
        # summarize performance
        print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))
        

def test_discriminator(model, test_case_sequence):
  X, _ = test_case_sequence_2_df([test_case_sequence])
  return model.predict(X)

In [16]:
%%time
# define the discriminator model
model = define_discriminator()

# fit the model
train_discriminator(model, X_train)

2023-07-05 06:56:38.997662: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:56:39.573733: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]
	 [[{{node Placeholder/_1}}]]


>1 real=27% fake=0%
>2 real=96% fake=27%
>3 real=88% fake=60%
>4 real=85% fake=77%
>5 real=75% fake=84%
>6 real=72% fake=84%


2023-07-05 06:56:39.896340: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:56:39.915137: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:56:39.935048: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]


>7 real=80% fake=85%
>8 real=78% fake=89%
>9 real=76% fake=87%
>10 real=72% fake=87%
>11 real=78% fake=88%
>12 real=78% fake=91%


2023-07-05 06:56:40.312770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:56:40.330578: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:56:40.347332: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]

...

2023-07-05 06:56:42.824837: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:56:42.841958: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:56:42.859068: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]

>86 real=100% fake=100%
>87 real=100% fake=99%
>88 real=100% fake=100%
>89 real=100% fake=99%
>90 real=100% fake=100%
>91 real=100% fake=100%


2023-07-05 06:56:43.026687: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:56:43.043363: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:56:43.060492: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]


>92 real=100% fake=100%
>93 real=100% fake=100%
>94 real=100% fake=99%
>95 real=100% fake=100%
>96 real=100% fake=99%
>97 real=100% fake=100%
>98 real=100% fake=100%
>99 real=100% fake=100%
>100 real=100% fake=99%
CPU times: user 5.02 s, sys: 2.4 s, total: 7.42 s
Wall time: 4.43 s


2023-07-05 06:56:43.236626: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:56:43.253305: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:56:43.269973: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]


In [17]:
test_discriminator(model, cr_raw_data_list[1])

1/1 [==============================] - 0s 97ms/step


array([[0.99317825]], dtype=float32)

In [18]:
test_discriminator(model, ['A', 'G', 'A', 'G', 'G'])

1/1 [==============================] - 0s 26ms/step


array([[0.00010747]], dtype=float32)

In this case, the discriminator model learns to tell the difference between real and randomly generated test case sequence very quickly, in about 100 iterations. Now that we know how to define and train the discriminator model, we need to look at developing the generator model.

## <font color='darkblue'>How to Define and Use the Generator Model</font> ([back](#sect0))
<b><font size='3ptx'>The generator model is responsible for creating new, fake but plausible test case sequence.</font></b>

<b>It does this by taking a point from the latent space as input and outputting a square grayscale image.</b>

<b>The [latent space](https://en.wikipedia.org/wiki/Latent_space) is an arbitrarily defined vector space of Gaussian-distributed values, e.g. 100 dimensions</b>. It has no meaning, but by drawing points from this space randomly and providing them to the generator model during training, the generator model will assign meaning to the latent points and, in turn, the latent space, until, at the end of training, the latent vector space represents a compressed representation of the output space, MNIST images, that only the generator knows how to turn into plausible MNIST images.
* **Inputs**: Point in latent space, e.g. a 100 element vector of Gaussian random numbers.
* **Outputs**: Two-dimensional (Sequence length x Number of test case) of 5×7 pixels with values in `(0,1)`.

<br/>

<b><font color='darkred'>Note</font></b>: we don’t have to use a 100 element vector as input; it is a round number and widely used, but I would expect that 10, 50, or 500 would work just as well. 

<b>Developing a generator model requires that we transform a vector from the latent space with, 100 dimensions to a 2D array with 5×7 or 35 values.</b> There are a number of ways to achieve this but <b>there is one approach that has proven effective at deep convolutional generative adversarial networks. It involves two main elements</b>.

<b>The first is a Dense layer as the first hidden layer that has enough nodes to represent a low-resolution version of the output data</b>. Specifically, a half the size (<font color='brown'>one quarter the area</font>) of the output data would be 3×3 or 9 nodes.

We don’t just want one low-resolution version of the image; we want many parallel versions or interpretations of the input. This is a [pattern in convolutional neural networks](https://machinelearningmastery.com/review-of-architectural-innovations-for-convolutional-neural-networks-for-image-classification/) where we have many parallel filters resulting in multiple parallel activation maps, called feature maps, with different interpretations of the input. We want the same thing in reverse: many parallel versions of our output with different learned features that can be collapsed in the output layer into a final sequence of test cases. The model needs space to invent, create, or generate.

Therefore, the first hidden layer, the Dense, needs enough nodes for multiple low-resolution versions of our output data, such as 128:
```python
# foundation for 3x3 nodes
model.add(Dense(1 * 3 * 3, input_dim=100))
```

<br/>

The activations from these nodes can then be reshaped into something image-like to pass into a convolutional layer, such as 1 different 3×3 feature maps.
```python
model.add(Reshape((3, 3, 1)))
```

<br/>

The next major architectural innovation involves upsampling the low-resolution image to a higher resolution version of the image. There are two common ways to do this upsampling process, sometimes called deconvolution.

One way is to use an [**UpSampling2D**](https://keras.io/api/layers/reshaping_layers/up_sampling2d/) layer (<font color='brown'>like a reverse pooling layer</font>) followed by a normal [**Conv2D**](https://keras.io/api/layers/convolution_layers/convolution2d/) layer. The other and perhaps more modern way is to combine these two operations into a single layer, called a Conv2DTranspose. We will use this latter approach for our generator.

The [**Conv2DTranspose**](https://keras.io/api/layers/convolution_layers/convolution2d_transpose/) layer can be configured with a stride of (2×2) that will quadruple the area of the input feature maps (<font color='brown'>double their width and height dimensions</font>). It is also good practice to use a kernel size that is a factor of the stride (<font color='brown'>e.g. double</font>) to [avoid a checkerboard pattern](https://distill.pub/2016/deconv-checkerboard/) that can be observed when upsampling:
```python
# upsample to 5x7
model.add(Conv2DTranspose(1, (5,7), strides=(2,2), padding='same'))
```

<br/>

The <font color='blue'>define_generator()</font> function below implements this and defines the generator model.

<b><font color='darkred'>Note</font></b>: the generator model is not compiled and does not specify a loss function or optimization algorithm. This is because the generator is not trained directly. We will learn more about this in the next section

In [19]:
# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 7x7 image
    n_nodes = 1 * 5 * 7
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((5, 7, 1)))
    #model.add(Conv2DTranspose(1, (5, 7), strides=(2,2), padding='same'))
    model.add(Conv2D(1, (5, 7), activation='sigmoid', padding='same'))
    return model

We can summarize the model to help better understand the input and output shapes. The complete example is listed below.

In [20]:
# define the size of the latent space
latent_dim = 10

# define the generator model
model = define_generator(latent_dim)

# summarize the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 35)                385       
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 35)                0         
                                                                 
 reshape (Reshape)           (None, 5, 7, 1)           0         
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 7, 1)           36        
                                                                 
Total params: 421
Trainable params: 421
Non-trainable params: 0
_________________________________________________________________


Running the example summarizes the layers of the model and their output shape.

Nevertheless, we can demonstrate how to use it to generate samples. This is a helpful demonstration to understand the generator as just another model, and some of these elements will be useful later.

The first step is to generate new points in the latent space. We can achieve this by calling the [randn()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.randn.html) NumPy function for generating arrays of [random numbers drawn from a standard Gaussian](https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/).

The array of random numbers can then be reshaped into samples, that is n rows with 100 elements per row. The <font color='blue'>generate_latent_points()</font> function below implements this and generates the desired number of points in the latent space that can be used as input to the generator model.

In [21]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

Next, we can use the generated points as input to the generator model to generate new samples, then print the samples.

We can update the <font color='blue'>generate_fake_samples()</font> function from the previous section to take the generator model as an argument and use it to generate the desired number of samples by first calling the <font color='blue'>generate_latent_points()</font> function to generate the required number of points in latent space as input to the model.

The updated <font color='blue'>generate_fake_samples_from_generator()</font> function is listed below and returns both the generated samples and the associated class labels.

In [22]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples_from_generator(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

We can then plot the generated samples as we did the real test case sequences (`cr_raw_data_list`) in the first section by calling the `raw_data_2_test_case_sequence` function to output human readable test case sequence. The complete example of generating new test case sequence with the untrained generator model is listed below:

In [23]:
# size of the latent space
latent_dim = 10

# define the discriminator model
model = define_generator(latent_dim)

# generate samples
n_samples = 5
X, _ = generate_fake_samples_from_generator(model, latent_dim, n_samples)

for tc_seq in X:
    print(raw_data_2_test_case_sequence(tc_seq))

1/1 [==============================] - 0s 70ms/step
['C', 'D', 'A', 'B', 'G']
['C', 'D', 'A', 'D', 'G']
['B', 'E', 'D', 'E', 'C']
['D', 'F', 'B', 'E', 'B']
['D', 'D', 'F', 'F', 'D']


In [24]:
# train the composite model
def train_gan(gan_model, latent_dim, n_epochs=100, n_batch=256):
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare points in latent space as input for the generator
        x_gan = generate_latent_points(latent_dim, n_batch)
        
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        
        # update the generator via the discriminator's error
        gan_model.train_on_batch(x_gan, y_gan)

Instead, what is required is that we first update the discriminator model with real and fake samples, then update the generator via the composite model.

This requires combining elements from the <font color='blue'>train_discriminator()</font> function defined in the discriminator section above and the <font color='blue'>train_gan()</font> function defined above. It also requires that we enumerate over both epochs and batches within in an epoch.

The complete train function for updating the discriminator model and the generator (<font color='brown'>via the composite model</font>) is listed below. There are a few things to note in this model training function.

First, the number of batches within an epoch is defined by how many times the batch size divides into the training dataset. <b>We have a dataset size of 60K samples, so with rounding down, there are 234 batches per epoch</b>.

The discriminator model is updated once per batch by combining one half a batch of fake and real examples into a single batch via the [vstack()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.vstack.html) NumPy function. You could update the discriminator with each half batch separately (<font color='brown'>recommended for more complex datasets</font>) but combining the samples into a single batch will be faster over a long run, especially when training on GPU hardware.

Finally, <b>we report the loss each batch. It is critical to keep an eye on the loss over batches. The reason for this is that a crash in the discriminator loss indicates that the generator model has started generating rubbish examples that the discriminator can easily discriminate</b>.

Monitor the discriminator loss and expect it to hover around 0.5 to 0.8 per batch on this dataset. The generator loss is less critical and may hover between 0.5 and 2 or higher on this dataset. A clever programmer might even attempt to detect the crashing loss of the discriminator, halt, and then restart the training process.

In [25]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=len(cr_raw_data_list)):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples_from_generator(g_model, latent_dim, half_batch)
            
            # create training set for the discriminator            
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))

            # update discriminator model weights
            d_loss, _ = d_model.train_on_batch(X, y)
            
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))

            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)

            # summarize loss on this batch
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))

We almost have everything we need to develop a GAN for the MNIST handwritten digits dataset.

One remaining aspect is the evaluation of the model.

<a id='sect4'></a>
## <font color='darkblue'>How to Train the Generator Model</font> ([back](#sect0))
<b><font size='3ptx'>The weights in the generator model are updated based on the performance of the discriminator model.</font></b>

<b>When the discriminator is good at detecting fake samples, the generator is updated more, and when the discriminator model is relatively poor or confused when detecting fake samples, the generator model is updated less</b>.

This defines the zero-sum or adversarial relationship between these two models.

There may be many ways to implement this using the Keras API, but perhaps the simplest approach is to create a new model that combines the generator and discriminator models.

Specifically, a new GAN model can be defined that stacks the generator and discriminator such that the generator receives as input random points in the latent space and generates samples that are fed into the discriminator model directly, classified, and the output of this larger model can be used to update the model weights of the generator.

To be clear, we are not talking about a new third model, just a new logical model that uses the already-defined layers and weights from the standalone generator and discriminator models.

Only the discriminator is concerned with distinguishing between real and fake examples, therefore the discriminator model can be trained in a standalone manner on examples of each, as we did in the section on the discriminator model above.

The generator model is only concerned with the discriminator’s performance on fake examples. Therefore, we will mark all of the layers in the discriminator as not trainable when it is part of the GAN model so that they can not be updated and overtrained on fake examples.

When training the generator via this logical GAN model, there is one more important change. We want the discriminator to think that the samples output by the generator are real, not fake. Therefore, when the generator is trained as part of the GAN model, we will mark the generated samples as real (class 1).

### <font color='darkgreen'>Why would we want to do this?</font>
We can imagine that the discriminator will then classify the generated samples as not real (class 0) or a low probability of being real (<font color='brown'>0.3 or 0.5</font>). The backpropagation process used to update the model weights will see this as a large error and will update the model weights (<font color='brown'>i.e. only the weights in the generator</font>) to correct for this error, in turn making the generator better at generating good fake samples.

Let’s make this concrete:
* **Inputs**: Point in latent space, e.g. a 100 element vector of Gaussian random numbers.
* **Outputs**: Binary classification, likelihood the sample is real (<font color='brown'>or fake</font>).

<br/>

The <font color='blue'>define_gan()</font> function below takes as arguments the already-defined generator and discriminator models and creates the new logical third model subsuming these two models. The weights in the discriminator are marked as not trainable, which only affects the weights as seen by the GAN model and not the standalone discriminator model.

The GAN model then uses the same binary cross entropy loss function as the discriminator and the efficient Adam version of stochastic gradient descent with the learning rate of 0.0002 and momentum 0.5, recommended when training deep convolutional GANs.

In [26]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.05, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

Making the discriminator not trainable is a clever trick in the Keras API.

The <font color='violet'>trainable</font> property impacts the model after it is compiled. The discriminator model was compiled with trainable layers, therefore the model weights in those layers will be updated when the standalone model is updated via calls to the <font color='blue'>train_on_batch()</font> function.

The discriminator model was then marked as not trainable, added to the GAN model, and compiled. In this model, the model weights of the discriminator model are not trainable and cannot be changed when the GAN model is updated via calls to the <font color='blue'>train_on_batch()</font> function. This change in the trainable property does not impact the training of standalone discriminator model.

This behavior is described in the Keras API documentation here:
* [How can I “freeze” Keras layers?](https://keras.io/getting-started/faq/#how-can-i-freeze-keras-layers)

<br/>

The complete example of creating the discriminator, generator, and composite model is listed below:

In [27]:
# size of the latent space
latent_dim = 10

# create the discriminator
d_model = define_discriminator()

# create the generator
g_model = define_generator(latent_dim)

# create the gan
gan_model = define_gan(g_model, d_model)

# summarize gan model
gan_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 5, 7, 1)           421       
                                                                 
 sequential_4 (Sequential)   (None, 1)                 1409      
                                                                 
Total params: 1,830
Trainable params: 421
Non-trainable params: 1,409
_________________________________________________________________


Running the example first creates a summary of the composite model.

We can see that <b>the model expects one sequence of test cases as input and predict a single value as output.</b>

Training the composite model involves generating a batch worth of points in the latent space via the <font color='blue'>generate_latent_points()</font> function in the previous section, and `class=1` labels and calling the <font color='blue'>train_on_batch()</font> function.

The <font color='blue'>train_gan()</font> function below demonstrates this, although is pretty simple as only the generator will be updated each epoch, leaving the discriminator with default model weights:

In [28]:
# train the composite model
def train_gan(gan_model, latent_dim, n_epochs=100, n_batch=256):
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare points in latent space as input for the generator
        x_gan = generate_latent_points(latent_dim, n_batch)
        
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        
        # update the generator via the discriminator's error
        gan_model.train_on_batch(x_gan, y_gan)

Instead, what is required is that we first update the discriminator model with real and fake samples, then update the generator via the composite model.

This requires combining elements from the <font color='blue'>train_discriminator()</font> function defined in the discriminator section above and the <font color='blue'>train_gan()</font> function defined above. It also requires that we enumerate over both epochs and batches within in an epoch.

The complete train function for updating the discriminator model and the generator (<font color='brown'>via the composite model</font>) is listed below. There are a few things to note in this model training function.

First, the number of batches within an epoch is defined by how many times the batch size divides into the training dataset. <b>We have a dataset size of 60K samples, so with rounding down, there are 234 batches per epoch</b>.

The discriminator model is updated once per batch by combining one half a batch of fake and real examples into a single batch via the [vstack()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.vstack.html) NumPy function. You could update the discriminator with each half batch separately (<font color='brown'>recommended for more complex datasets</font>) but combining the samples into a single batch will be faster over a long run, especially when training on GPU hardware.

Finally, <b>we report the loss each batch. It is critical to keep an eye on the loss over batches. The reason for this is that a crash in the discriminator loss indicates that the generator model has started generating rubbish examples that the discriminator can easily discriminate</b>.

Monitor the discriminator loss and expect it to hover around 0.5 to 0.8 per batch on this dataset. The generator loss is less critical and may hover between 0.5 and 2 or higher on this dataset. A clever programmer might even attempt to detect the crashing loss of the discriminator, halt, and then restart the training process.

In [31]:
# train the generator and discriminator
def train(
  g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=len(cr_raw_data_list),
  summarize_performance=None):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    real_batch = int(n_batch / 3)
    fake_batch = n_batch - real_batch
    # manually enumerate epochs
    for i in range(1, n_epochs):
      # enumerate batches over the training set
      for j in range(bat_per_epo):
        # get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(dataset, real_batch)
        # generate 'fake' examples
        X_fake, y_fake = generate_fake_samples_from_generator(g_model, latent_dim, fake_batch)
            
        # create training set for the discriminator            
        X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))

        # update discriminator model weights
        d_loss, _ = d_model.train_on_batch(X, y)
            
        # prepare points in latent space as input for the generator
        X_gan = generate_latent_points(latent_dim, n_batch)
            
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))

        # update the generator via the discriminator's error
        g_loss = gan_model.train_on_batch(X_gan, y_gan)

        # summarize loss on this batch
        print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
            
      if summarize_performance and i % 100 == 0:
        summarize_performance(
            i, g_model, d_model, dataset, latent_dim, n_samples=100)
        
    summarize_performance(
            -1, g_model, d_model, dataset, latent_dim, n_samples=100)

We almost have everything we need to develop a GAN for the MNIST handwritten digits dataset.

One remaining aspect is the evaluation of the model.

<a id='sect5'></a>
## <font color='darkblue'>How to Evaluate GAN Model Performance</font> ([back](#sect0))
<b><font size='3ptx'>Generally, there are no objective ways to evaluate the performance of a GAN model.</font></b>

We cannot calculate this objective error score for generated images. It might be possible in the case of MNIST images because the images are so well constrained, but in general, it is not possible (yet).

Instead, images must be subjectively evaluated for quality by a human operator. This means that we cannot know when to stop training without looking at examples of generated images. In turn, the adversarial nature of the training process means that the generator is changing after every batch, meaning that once “good enough” images can be generated, the subjective quality of the images may then begin to vary, improve, or even degrade with subsequent updates.

There are three ways to handle this complex training situation.
1. Periodically evaluate the classification accuracy of the discriminator on real and fake images.
2. Periodically generate many images and save them to file for subjective review.
3. Periodically save the generator model.

All three of these actions can be performed at the same time for a given training epoch, such as every five or 10 training epochs. The result will be a saved generator model for which we have a way of subjectively assessing the quality of its output and objectively knowing how well the discriminator was fooled at the time the model was saved.

Training the GAN over many epochs, such as hundreds or thousands of epochs, will result in many snapshots of the model that can be inspected and from which specific outputs and models can be cherry-picked for later use.

First, we can define a function called <font color='blue'>summarize_performance()</font> function that will summarize the performance of the discriminator model. It does this by retrieving a sample of real MNIST images, as well as generating the same number of fake MNIST images with the generator model, then evaluating the classification accuracy of the discriminator model on each sample and reporting these scores.

In [32]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)

    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)

    # prepare fake examples
    x_fake, y_fake = generate_fake_samples_from_generator(g_model, latent_dim, n_samples)

    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)

    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))

This function can be called from the <font color='blue'>train()</font> function based on the current epoch number, such as every 10 epochs:
```python
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
	  ...
	  # evaluate the model performance, sometimes
	  if (i+1) % 10 == 0:
		summarize_performance(i, g_model, d_model, dataset, latent_dim)
```

<br/>

Next, we can update the <font color='blue'>summarize_performance()</font> function to both save the model and to create and save a plot generated examples.

The generator model can be saved by calling the <font color='blue'>save()</font> function on the generator model and providing a unique filename based on the training epoch number.
```python
...
# save the generator model tile file
filename = 'generator_model_%03d.h5' % (epoch + 1)
g_model.save(filename)
```

<br/>

The updated <font color='blue'>summarize_performance()</font> function with these additions is listed below.

In [33]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)

    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)

    # prepare fake examples
    x_fake, y_fake = generate_fake_samples_from_generator(g_model, latent_dim, n_samples)

    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)

    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))

    # save plot
    # save_plot(x_fake, epoch)

    epoch_info = '%03d' % (epoch) if epoch > 0 else 'final'
    # save the generator model tile file
    filename = f'generator_sst_model_{epoch_info}.h5'
    g_model.save(filename)

<a id='sect6'></a>
## <font color='darkblue'>Complete Example of GAN for MNIST</font> ([back](#sect0))
<b><font size='3ptx'>We now have everything we need to train and evaluate a GAN on the MNIST handwritten digit dataset.</font></b>

For help on setting up an AWS EC2 instance to run this code, see the tutorial:
* [How to Setup Amazon AWS EC2 GPUs to Train Keras Deep Learning Models (step-by-step)](https://machinelearningmastery.com/develop-evaluate-large-deep-learning-models-keras-amazon-web-services/)

The complete example is listed below:

In [34]:
# size of the latent space
latent_dim = 10

# create the discriminator
d_model = define_discriminator()

# create the generator = define_generator(latent_dim)

# create the gan
gan_model = define_gan(g_model, d_model)

# load image data
dataset = X_train

In [35]:
# fit the model
train_discriminator(d_model, X_train)

2023-07-05 06:57:27.475462: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:27.945778: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]
	 [[{{node Placeholder/_1}}]]


>1 real=49% fake=0%
>2 real=100% fake=7%
>3 real=89% fake=49%
>4 real=82% fake=78%
>5 real=73% fake=80%
>6 real=75% fake=86%


2023-07-05 06:57:28.259209: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:28.278201: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:28.297464: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]


>7 real=84% fake=80%
>8 real=75% fake=84%
>9 real=75% fake=90%
>10 real=80% fake=84%
>11 real=81% fake=89%
>12 real=85% fake=91%


2023-07-05 06:57:28.669450: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:28.687133: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:28.707915: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]


...

2023-07-05 06:57:31.406902: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:31.424122: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:31.442623: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]


>91 real=100% fake=99%
>92 real=100% fake=100%
>93 real=100% fake=100%
>94 real=100% fake=100%
>95 real=100% fake=99%
>96 real=100% fake=100%


2023-07-05 06:57:31.622265: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:31.639023: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:31.658321: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]


>97 real=100% fake=100%
>98 real=100% fake=100%
>99 real=100% fake=99%
>100 real=100% fake=100%


2023-07-05 06:57:31.841348: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [85,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:31.857955: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [171,1]
	 [[{{node Placeholder/_1}}]]


In [36]:
%%time
# train model
train(
  g_model, d_model, gan_model, dataset, latent_dim, n_epochs=500,
  summarize_performance=summarize_performance)

1/1 [==============================] - 0s 68ms/step


2023-07-05 06:57:34.447920: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [27,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:34.715751: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [27,1]
	 [[{{node Placeholder/_1}}]]


>2, 1/1, d=0.003, g=21.329
1/1 [==============================] - 0s 28ms/step
>3, 1/1, d=0.009, g=17.134
1/1 [==============================] - 0s 26ms/step
>4, 1/1, d=0.002, g=13.953
1/1 [==============================] - 0s 27ms/step


2023-07-05 06:57:35.299860: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [27,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:35.313918: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [27,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:57:35.399766: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [27,1]
	

>5, 1/1, d=0.021, g=10.172
1/1 [==============================] - 0s 26ms/step
>6, 1/1, d=0.023, g=7.612
1/1 [==============================] - 0s 27ms/step


...

>497, 1/1, d=0.410, g=2.126
1/1 [==============================] - 0s 26ms/step
>498, 1/1, d=0.375, g=1.682
1/1 [==============================] - 0s 28ms/step


2023-07-05 06:58:28.502517: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [27,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:58:28.515989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [27,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:58:28.602639: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [27,1]
	

>499, 1/1, d=0.436, g=1.337
1/1 [==============================] - 0s 26ms/step
>500, 1/1, d=0.556, g=1.390


2023-07-05 06:58:28.703511: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [27,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:58:28.717891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [27,1]
	 [[{{node Placeholder/_1}}]]
2023-07-05 06:58:28.803427: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [27,1]
	

4/4 [==============================] - 0s 2ms/step
>Accuracy real: 52%, fake: 90%
CPU times: user 53 s, sys: 16.4 s, total: 1min 9s
Wall time: 54.8 s


The chosen configuration results in the stable training of both the generative and discriminative model.

The model performance is reported every batch, including the loss of both the discriminative (d) and generative (g) models.

<b><font color='darkred'>Note</font></b>: Your results may vary given the stochastic nature of the algorithm or evaluation procedure, or differences in numerical precision. Consider running the example a few times and compare the average outcome.

The generator is evaluated every 20 epochs, resulting in 10 evaluations, 10 plots of generated images, and 10 saved models.

In this case, we can see that the accuracy fluctuates over training. When viewing the discriminator model’s accuracy score in concert with generated images, we can see that the accuracy on fake examples does not correlate well with the subjective quality of images, but the accuracy for real examples may.

More training, beyond some point, does not mean better quality generated images.

In [37]:
# size of the latent space
latent_dim = 10

# generate samples
n_samples = 15
X, _ = generate_fake_samples_from_generator(g_model, latent_dim, n_samples)

for tc_seq in X:
    print(raw_data_2_test_case_sequence(tc_seq))

1/1 [==============================] - 0s 31ms/step
['A', 'B', 'D', 'E', 'G']
['E', 'F', 'D', 'C', 'D']
['C', 'B', 'C', 'A', 'B']
['D', 'A', 'B', 'C', 'F']
['F', 'F', 'D', 'E', 'G']
['C', 'F', 'D', 'A', 'B']
['A', 'B', 'C', 'E', 'G']
['D', 'E', 'B', 'C', 'G']
['A', 'B', 'C', 'E', 'G']
['D', 'A', 'B', 'C', 'G']
['C', 'E', 'G', 'A', 'B']
['D', 'A', 'G', 'A', 'G']
['A', 'F', 'C', 'E', 'G']
['A', 'B', 'C', 'E', 'G']
['A', 'B', 'C', 'E', 'G']


<a id='sect7'></a>
## <font color='darkblue'>How to Use the Final Generator Model to Generate Images</font> ([back](#sect0))
<b><font size='3ptx'>Once a final generator model is selected, it can be used in a standalone manner for your application.</font></b>

This involves first loading the model from file, then using it to generate test case sequences. The generation of each test sequence requires a point in the latent space as input.

The complete example of loading the saved model and generating test case sequence is listed below. In this case, we will use the model saved after 500 training epochs, but the model saved after 100 or 200 epochs would work just as well:

In [40]:
# example of loading the generator model and generating images
from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot

# generate points in latent space as input for the generator
def generate_latent_points(n_samples, latent_dim=10):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)

    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    
    return x_input


# load model
model = load_model('generator_sst_model_final.h5')

# generate images
latent_points = generate_latent_points(15)

# generate images
X = model.predict(latent_points)

1/1 [==============================] - 0s 72ms/step


In order to evaluate how good the generated test case squences are, we prepare come checking point found by association rules to see how similar those generated test case sequences from original dataset:

In [51]:
class ARule:
  def __init__(self, rule_str):
    self._order_mater = False
    self._elements = []
    self._orig_rule_str = rule_str
    if '->' in rule_str:
      self.order_mater = True
      self._elements = list(map(lambda e: e.strip(), rule_str.split('->')))
    else:
      self._elements = list(map(lambda e: e.strip(), rule_str.split(',')))
      
  def is_hit(self, test_sequence: list[str]):
    if self._order_mater:
      element_sequence_length = len(self._elements)
      for i in range(0, len(test_sequence) - element_sequence_length + 1):
        if test_sequence[i:i+element_sequence_length] == self._elements:
          return True
    else:
      element_set = set(self._elements)
      return len(element_set & set(test_sequence)) ==  len(element_set)
    
    return False
  
  def __str__(self):
    return f'ARule({self._orig_rule_str})'

In [54]:
rule_list = [
  ARule('A->B->C'), ARule('A,B,C'), ARule('F->D'), 
  ARule('C,E,G'), ARule('D,A,B'), ARule('F,D'),
  ARule('D,C'), ARule('E->G'), ARule('C,F'),
]

def hit_rule_count(test_sequence):
  hit_rule_count = 0
  return ', '.join([str(rule) for rule in rule_list if rule.is_hit(test_sequence)]) 

In [55]:
for tc_seq in X:
  test_sequence = raw_data_2_test_case_sequence(tc_seq)
  print(f'{test_sequence} ({hit_rule_count(test_sequence)})')

['F', 'F', 'D', 'E', 'G'] (ARule(F->D), ARule(F,D), ARule(E->G))
['E', 'A', 'D', 'C', 'A'] (ARule(D,C))
['E', 'A', 'B', 'C', 'F'] (ARule(A->B->C), ARule(A,B,C), ARule(C,F))
['F', 'F', 'D', 'E', 'G'] (ARule(F->D), ARule(F,D), ARule(E->G))
['C', 'A', 'C', 'A', 'B'] (ARule(A->B->C), ARule(A,B,C))
['A', 'B', 'C', 'D', 'G'] (ARule(A->B->C), ARule(A,B,C), ARule(D,A,B), ARule(D,C))
['D', 'A', 'B', 'C', 'F'] (ARule(A->B->C), ARule(A,B,C), ARule(F->D), ARule(D,A,B), ARule(F,D), ARule(D,C), ARule(C,F))
['C', 'F', 'D', 'A', 'G'] (ARule(F->D), ARule(F,D), ARule(D,C), ARule(C,F))
['C', 'C', 'G', 'A', 'B'] (ARule(A->B->C), ARule(A,B,C))
['D', 'E', 'B', 'C', 'F'] (ARule(F->D), ARule(F,D), ARule(D,C), ARule(C,F))
['D', 'E', 'B', 'C', 'F'] (ARule(F->D), ARule(F,D), ARule(D,C), ARule(C,F))
['A', 'B', 'D', 'D', 'G'] (ARule(D,A,B))
['C', 'B', 'C', 'A', 'G'] (ARule(A->B->C), ARule(A,B,C))
['G', 'E', 'G', 'A', 'B'] (ARule(E->G))
['A', 'B', 'C', 'D', 'F'] (ARule(A->B->C), ARule(A,B,C), ARule(F->D), ARule(D,A